In [1]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install XlsxWriter
!pip install qiskit
!pip install qiskit_optimization
!pip install networkx
!pip install --upgrade pip

In [4]:
%matplotlib inline

# useful packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx
import time
import pandas as pd

from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit.quantum_info import Statevector
from qiskit import Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_state_city, plot_bloch_multivector
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.visualization import plot_state_qsphere
import qiskit

In [5]:
# graph generation

def draw_graph(G, colors, pos):
        default_axes = plt.axes(frameon=True)
        nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

def graph_generation(dim):
    # Generating the system
    N = dim
    G1 = nx.grid_2d_graph(N,N)
    pos = dict( (n, n) for n in G1.nodes() )
    labels = dict( ((i, j), i * N + j) for i, j in G1.nodes() )
    # Transform to weighted graph:
    n = N**2
    V = np.arange(0, N, 1)

    E =[]

    tuples = []
    other_tup = []
    point_edge_map = []
    other_map = []
    relations = []

    for edge in G1.edges:
        point1 = edge[0]
        strpoint1 = str(point1[0]) + str(point1[1])
        point2 = edge[1]
        strpoint2 = str(point2[0]) + str(point2[1])
        if not strpoint1 in point_edge_map:
            point_edge_map.append(strpoint1)
            tuples.append(edge[0])
        if not strpoint2 in other_map:
            other_map.append(strpoint2)
            other_tup.append(edge[1])
        relations.append((strpoint1, strpoint2))

    tuples.append(other_tup[len(other_tup) - 1])
    point_edge_map.append(other_map[len(other_map) - 1])

    dic = {}
    for i in range(len(point_edge_map)):
        dic[i] = point_edge_map[i]

    point_edge_map = np.array(point_edge_map)
    relations = np.array(relations)

    for relation in relations:
        ver1 = relation[0]
        ver2 = relation[1]
        point1 = np.where(point_edge_map == ver1)[0][0]
        point2 = np.where(point_edge_map == ver2)[0][0]
        edge = (point1, point2, 1.0)
        E.append(edge)

    G = nx.Graph()

    G.add_nodes_from(V)

    G.add_weighted_edges_from(E)

    colors = ["b" for node in G.nodes()]
    pos = dict()
    for i in range(n):
        pos[i] = tuples[i]


    # draw_graph(G, colors, pos)
    
    # Matrix Representation of Graph
    w = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            temp = G.get_edge_data(i, j, default=0)
            if temp != 0:
                w[i, j] = temp["weight"]
                
    return G, w, pos

In [6]:
# output probability
def output_prob(parameters, size, rep): 
    '''
    parameters from vqe, size should be (number of interations, size**2 * rep)
    size: size of the ising model 
    rep: number of rep in the ansatz
    '''
    ### assign parameter in the circuit 
    num_q = size**2
    qc = QuantumCircuit(num_q)
    for i in range(rep + 1):
        for j in range(num_q): 
            qc.ry(parameters[j+i*num_q], j)
    
    ### measure 
    final_counts = backend.run(qc, shot = 1000).result().get_statevector()
    probs = Statevector(final_counts).probabilities()
    # coeff is the square root of probs 
    coeff = np.sqrt(probs)
    
    # function for writing wave function in binary form
    def zero(a,num_q):
        string = ''
        if len(a) < num_q:
            for _ in range(num_q-len(a)):
                string += '0'
            string += a
            return string
        # add an else statement
        else:
            return a

    info = {}
    # the largest integer that format() can convert to binary is 255.
    for i in range(len(coeff)):
        info[zero(bin(i)[2:], num_q)] = np.round(coeff[i], 3)
    return info

In [7]:
%%time
backend = Aer.get_backend('statevector_simulator')
# seed = 250
# algorithm_globals.random_seed = seed
# quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

iterations = np.linspace(400, 400, 1, dtype = 'int64')
seeds = np.linspace(201, 1000, 800, dtype = 'int64')

writer = pd.ExcelWriter('2x2 seeds 1.xlsx', engine = 'xlsxwriter')
for maxiter in iterations:
    G, w, pos = graph_generation(2)
    max_cut = Maxcut(w)
    qp = max_cut.to_quadratic_program()
    print('qp: ', qp)
    qubitOp, offset = qp.to_ising()
    print('qubitOp: ', qubitOp)
    exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

    data = pd.DataFrame()
    size = np.full_like(seeds, 4)
    optimizER = "COBYLA(maxiter=" + str(maxiter) + ")"
    opt = []
    first_state = []
    first_prob = []
    first_energy = []
    terminate_it = []
    energy = []
    time = []
    max_cum_objective = []
    solution = []
    solution_objective = []
    
    for i, seed in enumerate(seeds):
        algorithm_globals.random_seed = seed
        quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)
        
        reps = 1
        # construct VQE
        cobyla = COBYLA(maxiter = maxiter)
        ry = TwoLocal(qubitOp.num_qubits, "ry", reps = reps, entanglement="linear")
        # monitoring VQE
        counts, values, Parameters = [], [], []
        def store_intermediate_result(eval_count, parameter, mean, std):
            counts.append(eval_count)
            values.append(mean)
            Parameters.append(parameter)
        
        vqe = VQE(ry, optimizer=cobyla, callback=store_intermediate_result, quantum_instance=quantum_instance)
        
        # run VQE
        result = vqe.compute_minimum_eigenvalue(qubitOp)
        x = max_cut.sample_most_likely(result.eigenstate)
        
        # state after the first iteration
        first_it = output_prob(Parameters[0], 2, reps)
        print(first_it)
        f_prob, f_state = max(first_it.values()), max(first_it, key = first_it.get)
        
        # data collection
        first_state.append(f_state)
        first_prob.append(f_prob)
        first_energy.append(values[0])
        terminate_it.append(len(Parameters))
        opt.append(optimizER)
        energy.append(result.eigenvalue.real)
        time.append(result.optimizer_time)
        max_cum_objective.append(result.eigenvalue.real + offset)
        solution.append(max_cut.sample_most_likely(result.eigenstate))
        solution_objective.append(qp.objective.evaluate(x))

    # data base
    data['seed'] = seeds
    data['size'] = size
    data['optimizer'] = opt
    data['state after 1st iteration'] = first_state
    data['probability for the 1st state'] = first_prob
    data['energy for the 1st state'] = first_energy
    data['terminate iterations'] = terminate_it
    data['energy'] = energy
    data['time (s)'] = time
    data['max cut objective'] = max_cum_objective
    data['solution'] = solution
    data['solution objective'] = solution_objective
    data.to_excel(writer, sheet_name = "maxiter=" + str(maxiter))
    
writer.save()

qp:  maximize -2*x_0*x_1 - 2*x_0*x_2 - 2*x_1*x_3 - 2*x_2*x_3 + 2*x_0 + 2*x_1 + 2*x_2 + 2*x_3 (4 variables, 0 constraints, 'Max-cut')
qubitOp:  0.5 * IIZZ
+ 0.5 * IZIZ
+ 0.5 * ZIZI
+ 0.5 * ZZII
{'0000': 0.181, '0001': 0.353, '0010': 0.067, '0011': 0.131, '0100': 0.218, '0101': 0.426, '0110': 0.08, '0111': 0.157, '1000': 0.204, '1001': 0.4, '1010': 0.075, '1011': 0.148, '1100': 0.246, '1101': 0.481, '1110': 0.091, '1111': 0.178}
{'0000': 0.323, '0001': 0.426, '0010': 0.289, '0011': 0.381, '0100': 0.306, '0101': 0.404, '0110': 0.274, '0111': 0.361, '1000': 0.05, '1001': 0.065, '1010': 0.044, '1011': 0.058, '1100': 0.047, '1101': 0.062, '1110': 0.042, '1111': 0.055}
{'0000': 0.029, '0001': 0.021, '0010': 0.104, '0011': 0.073, '0100': 0.102, '0101': 0.072, '0110': 0.363, '0111': 0.256, '1000': 0.053, '1001': 0.038, '1010': 0.19, '1011': 0.134, '1100': 0.186, '1101': 0.132, '1110': 0.663, '1111': 0.469}
{'0000': 0.42, '0001': 0.127, '0010': 0.036, '0011': 0.011, '0100': 0.856, '0101': 0.259,

{'0000': 0.548, '0001': 0.432, '0010': 0.295, '0011': 0.233, '0100': 0.382, '0101': 0.302, '0110': 0.206, '0111': 0.162, '1000': 0.145, '1001': 0.115, '1010': 0.078, '1011': 0.062, '1100': 0.101, '1101': 0.08, '1110': 0.055, '1111': 0.043}
{'0000': 0.052, '0001': 0.359, '0010': 0.032, '0011': 0.221, '0100': 0.01, '0101': 0.069, '0110': 0.006, '0111': 0.042, '1000': 0.107, '1001': 0.745, '1010': 0.066, '1011': 0.46, '1100': 0.021, '1101': 0.143, '1110': 0.013, '1111': 0.088}
{'0000': 0.004, '0001': 0.012, '0010': 0.007, '0011': 0.021, '0100': 0.157, '0101': 0.461, '0110': 0.276, '0111': 0.812, '1000': 0.001, '1001': 0.002, '1010': 0.001, '1011': 0.003, '1100': 0.026, '1101': 0.077, '1110': 0.046, '1111': 0.136}
{'0000': 0.036, '0001': 0.737, '0010': 0.003, '0011': 0.063, '0100': 0.027, '0101': 0.556, '0110': 0.002, '0111': 0.048, '1000': 0.015, '1001': 0.297, '1010': 0.001, '1011': 0.026, '1100': 0.011, '1101': 0.224, '1110': 0.001, '1111': 0.019}
{'0000': 0.754, '0001': 0.506, '0010': 

{'0000': 0.519, '0001': 0.083, '0010': 0.655, '0011': 0.105, '0100': 0.308, '0101': 0.05, '0110': 0.389, '0111': 0.063, '1000': 0.091, '1001': 0.015, '1010': 0.115, '1011': 0.019, '1100': 0.054, '1101': 0.009, '1110': 0.069, '1111': 0.011}
{'0000': 0.073, '0001': 0.142, '0010': 0.195, '0011': 0.378, '0100': 0.002, '0101': 0.004, '0110': 0.006, '0111': 0.011, '1000': 0.143, '1001': 0.278, '1010': 0.381, '1011': 0.741, '1100': 0.004, '1101': 0.008, '1110': 0.011, '1111': 0.022}
{'0000': 0.122, '0001': 0.204, '0010': 0.123, '0011': 0.205, '0100': 0.012, '0101': 0.021, '0110': 0.013, '0111': 0.021, '1000': 0.339, '1001': 0.565, '1010': 0.342, '1011': 0.57, '1100': 0.035, '1101': 0.058, '1110': 0.035, '1111': 0.058}
{'0000': 0.608, '0001': 0.179, '0010': 0.591, '0011': 0.174, '0100': 0.04, '0101': 0.012, '0110': 0.039, '0111': 0.012, '1000': 0.318, '1001': 0.094, '1010': 0.31, '1011': 0.091, '1100': 0.021, '1101': 0.006, '1110': 0.021, '1111': 0.006}
{'0000': 0.116, '0001': 0.285, '0010': 0

{'0000': 0.0, '0001': 0.0, '0010': 0.001, '0011': 0.002, '0100': 0.012, '0101': 0.014, '0110': 0.044, '0111': 0.05, '1000': 0.006, '1001': 0.007, '1010': 0.021, '1011': 0.023, '1100': 0.179, '1101': 0.203, '1110': 0.635, '1111': 0.719}
{'0000': 0.01, '0001': 0.004, '0010': 0.01, '0011': 0.004, '0100': 0.109, '0101': 0.046, '0110': 0.106, '0111': 0.044, '1000': 0.059, '1001': 0.025, '1010': 0.057, '1011': 0.024, '1100': 0.651, '1101': 0.273, '1110': 0.63, '1111': 0.264}
{'0000': 0.319, '0001': 0.042, '0010': 0.798, '0011': 0.104, '0100': 0.024, '0101': 0.003, '0110': 0.061, '0111': 0.008, '1000': 0.182, '1001': 0.024, '1010': 0.455, '1011': 0.059, '1100': 0.014, '1101': 0.002, '1110': 0.035, '1111': 0.005}
{'0000': 0.021, '0001': 0.149, '0010': 0.07, '0011': 0.491, '0100': 0.018, '0101': 0.125, '0110': 0.059, '0111': 0.412, '1000': 0.023, '1001': 0.162, '1010': 0.076, '1011': 0.534, '1100': 0.019, '1101': 0.136, '1110': 0.064, '1111': 0.449}
{'0000': 0.02, '0001': 0.085, '0010': 0.024, 

{'0000': 0.038, '0001': 0.026, '0010': 0.054, '0011': 0.037, '0100': 0.007, '0101': 0.005, '0110': 0.01, '0111': 0.007, '1000': 0.468, '1001': 0.318, '1010': 0.663, '1011': 0.45, '1100': 0.085, '1101': 0.058, '1110': 0.12, '1111': 0.082}
{'0000': 0.405, '0001': 0.416, '0010': 0.326, '0011': 0.335, '0100': 0.268, '0101': 0.276, '0110': 0.216, '0111': 0.221, '1000': 0.203, '1001': 0.209, '1010': 0.163, '1011': 0.168, '1100': 0.134, '1101': 0.138, '1110': 0.108, '1111': 0.111}
{'0000': 0.227, '0001': 0.149, '0010': 0.214, '0011': 0.14, '0100': 0.432, '0101': 0.283, '0110': 0.406, '0111': 0.267, '1000': 0.17, '1001': 0.111, '1010': 0.16, '1011': 0.105, '1100': 0.322, '1101': 0.212, '1110': 0.303, '1111': 0.199}
{'0000': 0.183, '0001': 0.052, '0010': 0.009, '0011': 0.003, '0100': 0.883, '0101': 0.254, '0110': 0.045, '0111': 0.013, '1000': 0.066, '1001': 0.019, '1010': 0.003, '1011': 0.001, '1100': 0.321, '1101': 0.092, '1110': 0.016, '1111': 0.005}
{'0000': 0.15, '0001': 0.359, '0010': 0.04

...

{'0000': 0.084, '0001': 0.266, '0010': 0.024, '0011': 0.078, '0100': 0.219, '0101': 0.694, '0110': 0.064, '0111': 0.203, '1000': 0.06, '1001': 0.191, '1010': 0.018, '1011': 0.056, '1100': 0.157, '1101': 0.499, '1110': 0.046, '1111': 0.146}
{'0000': 0.198, '0001': 0.01, '0010': 0.023, '0011': 0.001, '0100': 0.24, '0101': 0.012, '0110': 0.028, '0111': 0.001, '1000': 0.6, '1001': 0.03, '1010': 0.069, '1011': 0.004, '1100': 0.726, '1101': 0.037, '1110': 0.084, '1111': 0.004}
{'0000': 0.655, '0001': 0.432, '0010': 0.227, '0011': 0.15, '0100': 0.423, '0101': 0.279, '0110': 0.146, '0111': 0.097, '1000': 0.101, '1001': 0.067, '1010': 0.035, '1011': 0.023, '1100': 0.065, '1101': 0.043, '1110': 0.023, '1111': 0.015}
{'0000': 0.376, '0001': 0.01, '0010': 0.214, '0011': 0.006, '0100': 0.002, '0101': 0.0, '0110': 0.001, '0111': 0.0, '1000': 0.783, '1001': 0.022, '1010': 0.446, '1011': 0.012, '1100': 0.005, '1101': 0.0, '1110': 0.003, '1111': 0.0}
{'0000': 0.256, '0001': 0.104, '0010': 0.182, '0011'

{'0000': 0.396, '0001': 0.65, '0010': 0.283, '0011': 0.465, '0100': 0.148, '0101': 0.242, '0110': 0.106, '0111': 0.174, '1000': 0.02, '1001': 0.033, '1010': 0.014, '1011': 0.023, '1100': 0.007, '1101': 0.012, '1110': 0.005, '1111': 0.009}
{'0000': 0.485, '0001': 0.804, '0010': 0.134, '0011': 0.222, '0100': 0.088, '0101': 0.145, '0110': 0.024, '0111': 0.04, '1000': 0.07, '1001': 0.116, '1010': 0.019, '1011': 0.032, '1100': 0.013, '1101': 0.021, '1110': 0.003, '1111': 0.006}
{'0000': 0.002, '0001': 0.047, '0010': 0.001, '0011': 0.034, '0100': 0.008, '0101': 0.2, '0110': 0.006, '0111': 0.146, '1000': 0.007, '1001': 0.177, '1010': 0.005, '1011': 0.13, '1100': 0.029, '1101': 0.759, '1110': 0.021, '1111': 0.556}
{'0000': 0.082, '0001': 0.11, '0010': 0.555, '0011': 0.743, '0100': 0.003, '0101': 0.004, '0110': 0.023, '0111': 0.03, '1000': 0.03, '1001': 0.041, '1010': 0.205, '1011': 0.274, '1100': 0.001, '1101': 0.002, '1110': 0.008, '1111': 0.011}
{'0000': 0.208, '0001': 0.285, '0010': 0.082, 

{'0000': 0.167, '0001': 0.003, '0010': 0.056, '0011': 0.001, '0100': 0.29, '0101': 0.005, '0110': 0.098, '0111': 0.002, '1000': 0.442, '1001': 0.008, '1010': 0.149, '1011': 0.003, '1100': 0.768, '1101': 0.014, '1110': 0.259, '1111': 0.005}
{'0000': 0.16, '0001': 0.092, '0010': 0.586, '0011': 0.336, '0100': 0.024, '0101': 0.014, '0110': 0.09, '0111': 0.051, '1000': 0.159, '1001': 0.091, '1010': 0.584, '1011': 0.335, '1100': 0.024, '1101': 0.014, '1110': 0.089, '1111': 0.051}
{'0000': 0.001, '0001': 0.001, '0010': 0.036, '0011': 0.048, '0100': 0.01, '0101': 0.013, '0110': 0.573, '0111': 0.761, '1000': 0.0, '1001': 0.0, '1010': 0.011, '1011': 0.015, '1100': 0.003, '1101': 0.004, '1110': 0.179, '1111': 0.238}
{'0000': 0.154, '0001': 0.3, '0010': 0.308, '0011': 0.6, '0100': 0.102, '0101': 0.198, '0110': 0.204, '0111': 0.396, '1000': 0.073, '1001': 0.142, '1010': 0.146, '1011': 0.284, '1100': 0.048, '1101': 0.094, '1110': 0.097, '1111': 0.188}
{'0000': 0.001, '0001': 0.013, '0010': 0.0, '001

{'0000': 0.197, '0001': 0.203, '0010': 0.449, '0011': 0.461, '0100': 0.158, '0101': 0.162, '0110': 0.36, '0111': 0.37, '1000': 0.095, '1001': 0.098, '1010': 0.217, '1011': 0.223, '1100': 0.076, '1101': 0.078, '1110': 0.174, '1111': 0.178}
{'0000': 0.184, '0001': 0.611, '0010': 0.071, '0011': 0.234, '0100': 0.074, '0101': 0.244, '0110': 0.028, '0111': 0.093, '1000': 0.17, '1001': 0.562, '1010': 0.065, '1011': 0.216, '1100': 0.068, '1101': 0.224, '1110': 0.026, '1111': 0.086}
{'0000': 0.012, '0001': 0.009, '0010': 0.031, '0011': 0.024, '0100': 0.197, '0101': 0.154, '0110': 0.514, '0111': 0.401, '1000': 0.012, '1001': 0.01, '1010': 0.032, '1011': 0.025, '1100': 0.201, '1101': 0.157, '1110': 0.526, '1111': 0.409}
{'0000': 0.436, '0001': 0.404, '0010': 0.579, '0011': 0.538, '0100': 0.063, '0101': 0.058, '0110': 0.083, '0111': 0.077, '1000': 0.02, '1001': 0.018, '1010': 0.026, '1011': 0.024, '1100': 0.003, '1101': 0.003, '1110': 0.004, '1111': 0.003}
{'0000': 0.143, '0001': 0.262, '0010': 0.

{'0000': 0.003, '0001': 0.001, '0010': 0.001, '0011': 0.0, '0100': 0.01, '0101': 0.002, '0110': 0.002, '0111': 0.0, '1000': 0.254, '1001': 0.051, '1010': 0.054, '1011': 0.011, '1100': 0.925, '1101': 0.184, '1110': 0.197, '1111': 0.039}
{'0000': 0.149, '0001': 0.143, '0010': 0.018, '0011': 0.017, '0100': 0.246, '0101': 0.236, '0110': 0.03, '0111': 0.029, '1000': 0.34, '1001': 0.326, '1010': 0.041, '1011': 0.039, '1100': 0.561, '1101': 0.539, '1110': 0.068, '1111': 0.065}
{'0000': 0.007, '0001': 0.009, '0010': 0.314, '0011': 0.397, '0100': 0.002, '0101': 0.002, '0110': 0.067, '0111': 0.084, '1000': 0.012, '1001': 0.015, '1010': 0.519, '1011': 0.656, '1100': 0.003, '1101': 0.003, '1110': 0.11, '1111': 0.14}
{'0000': 0.79, '0001': 0.174, '0010': 0.095, '0011': 0.021, '0100': 0.351, '0101': 0.077, '0110': 0.042, '0111': 0.009, '1000': 0.402, '1001': 0.088, '1010': 0.048, '1011': 0.011, '1100': 0.179, '1101': 0.039, '1110': 0.021, '1111': 0.005}
{'0000': 0.075, '0001': 0.1, '0010': 0.462, '0